# Analysis Part III - Continued

In [ ]:
%load_ext autoreload
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore')
import os
import scanpy as sc
import scirpy as ir
import anndata as ann
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from matplotlib import rcParams
from mudata import MuData
import mudata

import tarfile
import warnings
from glob import glob

import anndata
import matplotlib.pyplot as plt
import muon as mu
import pandas as pd
import scanpy as sc
import scirpy as ir

%autoreload 2
import sys
sys.path.append('..')
import utility.annotation as utils_annotation
import utility.representation as utils_representation
import utility.visualisation as utils_vis

from sklearn.mixture import GaussianMixture

from matplotlib.colors import LinearSegmentedColormap

In [ ]:
sc.settings.set_figure_params(dpi=150)
sc.settings.verbosity = 3
sc.set_figure_params(vector_friendly=True, color_map='viridis')
sb.set_style('whitegrid')

colormap = 'flare'

## Get input data

In [ ]:
mdata = mu.read("/media/agschober/HDD12/3_scRNA-Seq_Sina/3_Specificity/data_specificity_annotated_final.h5mu")

## Differentially expressed genes and cluster annotation

In [ ]:
def deg_over_condition(condition, adata):
    adata_tmp = utils_representation.filter_tcr_genes(adata)
    adata_tmp = utils_representation.filter_high_var(adata_tmp, 5000)
    sc.tl.rank_genes_groups(adata_tmp, groupby=condition, n_genes=20)
    sc.pl.rank_genes_groups(adata_tmp, groubpy=condition, show=False)
    plt.tight_layout()
    plt.show()
    adata.uns[f'rank_genes_groups_{condition}'] = adata_tmp.uns['rank_genes_groups']

In [ ]:
deg_over_condition('leiden', mdata["gex"])

## PseudoTime

In [ ]:
utils_representation.calculate_diffmap(mdata["gex"], n_high_var=5000, remove_tcr_genes=True)

In [ ]:
(mdata['gex'].obsm['X_diffmap'].shape[1])

In [ ]:
nrows = 20
ncols = 5
fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols * 3, nrows * 3))
axes = axes.reshape(-1)

for i, ax in zip(range(mdata['gex'].obsm['X_diffmap'].shape[1]), axes):
    root_ixs = mdata['gex'].obsm['X_diffmap'][:, i].argmin()
    root_umap = mdata['gex'].obsm['X_umap'][root_ixs]

    sc.pl.umap(mdata['gex'], show=False, title=str(i), ax=ax)
    ax.plot(root_umap[0], root_umap[1],  marker='o', markersize=5, color="red")
fig.tight_layout()
plt.show()

In [ ]:
colors = ['darkblue', 'blue', 'lightblue', '#FFEA00', 'orange', 'red', 'darkred']

cust = LinearSegmentedColormap.from_list('custom_cmap', colors)

In [ ]:
root_nr = 69
root_ixs = mdata['gex'].obsm['X_diffmap'][:, root_nr].argmin()
mdata['gex'].uns['iroot'] = root_ixs
utils_representation.calculate_dpt(mdata['gex'], n_high_var=5000, remove_tcr_genes=True)
mdata.obs['dpt_pseudotime'] = mdata['gex'].obs['dpt_pseudotime']
mdata["gex"].obs['dpt_pseudotime'] = mdata.obs['dpt_pseudotime']

In [ ]:
rcParams['figure.figsize'] = (6, 6)
sc.pl.umap(mdata["gex"], color='dpt_pseudotime', cmap=cust)
plt.tight_layout()
plt.savefig(f'/media/agschober/HDD12/3_scRNA-Seq_Sina/3_Specificity/Pseudotime_root_69.png')
plt.show()

In [ ]:
sc.pl.violin(mdata['gex'], keys='dpt_pseudotime', groupby='leiden')

In [ ]:
#get missing IR data
mdata["airr"].obs["IR_VJ_1_junction"] = ir.get.airr(mdata, "junction", ["VJ_1"])
mdata["airr"].obs["IR_VJ_2_junction"] = ir.get.airr(mdata, "junction", ["VJ_2"])
mdata["airr"].obs["IR_VDJ_1_junction"] = ir.get.airr(mdata, "junction", ["VDJ_1"])
mdata["airr"].obs["IR_VDJ_2_junction"] = ir.get.airr(mdata, "junction", ["VDJ_2"])

## Save the data

In [ ]:
mdata.write("/media/agschober/HDD12/3_scRNA-Seq_Sina/3_Specificity/data_specificity_annotated_final_pseudotime.h5mu")

In [ ]:
import session_info
session_info.show()